# Custom container with SageMaker

In [1]:
import os
import numpy as np
import sagemaker
import boto3
import time
import math
import json
import re
import pandas as pd
from sagemaker.session import s3_input
from sagemaker.estimator import Estimator

sagemaker_session = sagemaker.Session()

client = boto3.client('sagemaker')
region = boto3.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')
role = 'arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190503T215251'.format(account)

In [2]:
# Specify the dataset locations
train_path = 's3://deep-learning-container-webinar/cifar10-dataset'
eval_path = 's3://deep-learning-container-webinar/cifar10-dataset'
output_path = 's3://deep-learning-container-webinar/'

# Specify the container image to use
image = '{}.dkr.ecr.{}.amazonaws.com/dl-webinar-repo-sagemaker:latest'.format(account, region)

# Create a sagemaker estimator
estimator = Estimator(
    image_name=image,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.p3.2xlarge',
    output_path=output_path,
    sagemaker_session=sagemaker_session)

In [3]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
job_name = 'webinar-' + time.strftime('%Y-%m-%d-%H-%M-%S-%j', time.gmtime())

hyperparameter_ranges = {
    'max-learning-rate': ContinuousParameter(0.4, 0.9), 
    'data_aug_cutout_size': CategoricalParameter([5,8,10,12]),
    'batch-size':    CategoricalParameter([128, 512]),
    'batch_norm':    CategoricalParameter(['true', 'false']),
    'momentum':      ContinuousParameter(0.9, 0.99),
    'epochs':        CategoricalParameter([10, 15])
}

objective_metric_name = 'test_acc'
objective_type = 'Maximize'
metric_definitions=[{'Name': 'test_acc', 'Regex': 'test_acc:([0-9\\.]+)'}]

tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            strategy='Random',
                            max_jobs=16,
                            max_parallel_jobs=8,
                            objective_type=objective_type)

In [4]:
tuner.fit({'training': train_path,'eval': eval_path}, job_name=job_name)